# 🏥 Pharma-Safe Lens - Kaggle Validation Notebook

**Complete validation for all phases**

## Setup Instructions
1. **Phase 1-2**: CPU only (no GPU needed)
2. **Phase 3+**: Enable GPU accelerator

## Important: Run cells in order!

## Cell 0: Install System Dependencies (REQUIRED)

Tesseract OCR must be installed before Python packages.

In [ ]:
# Install Tesseract OCR engine
!apt-get update -y
!apt-get install -y tesseract-ocr

# Verify
!tesseract --version

## Cell 1: Clone Repository

In [ ]:
# Clone from GitHub (replace YOUR_USERNAME)
!git clone https://github.com/YOUR_USERNAME/pharma-safe-lens.git
%cd pharma-safe-lens

## Cell 2: Install Python Dependencies

In [ ]:
%cd backend
!pip install -r requirements.txt

## Cell 3: Verify Imports

In [ ]:
import sys
sys.path.insert(0, '/kaggle/working/pharma-safe-lens')

# Test imports
import easyocr
import pytesseract
import cv2
from backend.app.drug_db import DrugDatabase
from backend.app.ocr import extract_text

print("✅ All imports successful!")

## Phase 1 Validation: Drug Database

In [ ]:
# Test drug database
db = DrugDatabase()

print(f"Loaded {len(db.drug_map)} drugs")
print(f"\nTest exact match: aspirin -> {db.get_generic_name('aspirin')}")
print(f"Test brand name: ecosprin -> {db.get_generic_name('ecosprin')}")
print(f"Test fuzzy match: asprin -> {db.get_generic_name('asprin')}")

# Test normalization
raw_text = ['ASPIRIN 100MG', 'WARFARIN 5MG', 'MFG:2024']
result = db.normalize(raw_text)
print(f"\nNormalization test:")
print(f"  Input: {raw_text}")
print(f"  Output: {result}")

print("\n✅ Phase 1 Drug Database: PASSED")

## Phase 1 Validation: OCR (with uploaded image)

In [ ]:
# If you have uploaded a pill strip image, test OCR here
# Replace 'your-image.jpg' with your actual image path

# Example:
# image_path = '/kaggle/input/your-dataset/pill-strip.jpg'
# texts = extract_text(image_path)
# print(f"Extracted text: {texts}")
# 
# drugs = db.normalize(texts)
# print(f"Detected drugs: {drugs}")

print("Upload an image to test OCR functionality")

## Phase 2 Validation: Interaction Checker

*(Will be implemented in Phase 2)*

In [ ]:
# TODO: Phase 2 validation
print("Phase 2 not yet implemented")

## Phase 3 Validation: MedGemma

*(Will be implemented in Phase 3 - requires GPU)*

In [ ]:
# TODO: Phase 3 validation
print("Phase 3 not yet implemented")